In [ ]:
import numpy as np
import pandas as pd
#pandas字串操作
x=np.array([2,3,5,7,9,11])
print(x*2)
data=['peter','Paul','MARY','gUIDO']
print([s.capitalize() for s in data])
data=['peter','None','Paul','MARY','gUIDO']
names=pd.Series(data)
print(names)
print(names.str.capitalize())
#pandas字串方法表格
monte=pd.Series(['Graham Chapman','John Cleese','Terry Gilliam',
         'Eric Idle','Terry Jones','Michael Palin'])
print(monte)
print(monte.str.lower())#回傳小寫字串
print(monte.str.len())#字串長度
print(monte.str.startswith('T'))#輸出true or fale,開頭是否為t
print(monte.str.split())#傳回list或是元素複合值
print(monte.str.match('Terry'))#回傳是否有一樣的字串(一組一組看)
#正則表達法
print(monte.str.extract('([A-Za-z]+)'))
print(monte.str.findall(r'^[^AEIOU].*[^aeiou]$'))
print(monte.str.findall('^[^AEIOU].*[^aeiou]$'))
#雜項方法
print(monte.str.slice(0,3))
print(monte.str[0:3])
print(monte.str.split().str.get(-1))
#指示符變數
full_monte=pd.DataFrame({'info':['B|C|D','B|D','A|C','B|D',
                 'B|C','B|C|D'],'name':monte})
print(full_monte)
#print(full_monte['info'].str.get_dummies('|'))
new_full_monte=full_monte['info'].str.get_dummies('|')
new_full_monte['name']=monte
print(new_full_monte)

recipes=pd.read_json(StringIO(data_json))而不是recipes=pd.read_json(data_json)

It seems like read_json now looks for a file-like object rather than a string-like one. StringIO provides a file-like interface for strings. From the pandas docs: By file-like object, we refer to objects with a read() method, such as a file handler (e.g. via builtin open function) or StringIO


In [34]:
pip install fsspec

In [15]:
#範例:食譜資料庫
import numpy as np
import pandas as pd
from io import StringIO
#發現會出錯誤
try:
  recipes=pd.read_json('/content/drive/MyDrive/資料科學家/DS-Coding/python資料科學學習手冊/data/recipeitems-latest.json')
except ValueError as e:
  print("ValueError:",e)
##原因為整個檔案並非是JSON格式(但每一列都是)
with open('/content/drive/MyDrive/資料科學家/DS-Coding/python資料科學學習手冊/data/recipeitems-latest.json') as f:
  line=f.readline()
print(pd.read_json(StringIO(line)).shape)
#把所有串成一組字串
with open('/content/drive/MyDrive/資料科學家/DS-Coding/python資料科學學習手冊/data/recipeitems-latest.json','r') as f:
  data=(line.strip() for line in f)#提取每行並去掉頭尾空白
  data_json='[{}]'.format(','.join(data))#先用join每行以逗點隔開,再用.format存入{}最後會像個list
recipes=pd.read_json(StringIO(data_json))
print(recipes.shape)
#初步了解recipes長相
print(recipes.iloc[0])
#大部分資料雜,因為是從網站上擷取.例如:食材清單多為字串
#了解食材清單之字串分布
recipes.ingredients.str.len().describe()
#長度最長食材清單的食譜
print(recipes.name[np.argmax(recipes.ingredients.str.len())])
#多少筆為早餐用(從description中的描述)
print(recipes.description.str.contains('[Bb]reakfast').sum())
#使用"肉桂"這個食材
print(recipes.ingredients.str.contains('[Cc]innamon').sum())
#有可能將"肉桂"的英文拼錯
print(recipes.ingredients.str.contains('[Cc]inamon').sum())

ValueError: Trailing data
(2, 12)
(173278, 17)
_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator          

In [21]:
#簡單的食譜推薦器
import numpy as np
import pandas as pd
import re
#簡化問題,只考慮草本植物和香料
spice_list=['salt','papper','oregano','sage','parsley',
      'rosemary','tarragon','thyme','paprika','cumin']
#是否含有該食材
spice_df=pd.DataFrame(dict((spice,recipes.ingredients.str.contains(spice,re.IGNORECASE))for
                           spice in spice_list))
print(spice_df.head())
#更有效率的方法(用到parsley且paprika且tarraagon)的數量
selection=spice_df.query('parsley & paprika & tarragon')
print(selection)
print(len(selection))
#藉由selection索引找出食譜
print(recipes.name[selection.index])

    salt  papper  oregano   sage  ...  tarragon  thyme  paprika  cumin
0  False   False    False   True  ...     False  False    False  False
1  False   False    False  False  ...     False  False    False  False
2   True   False    False  False  ...     False  False    False   True
3  False   False    False  False  ...     False  False    False  False
4  False   False    False  False  ...     False  False    False  False

[5 rows x 10 columns]
         salt  papper  oregano   sage  ...  tarragon  thyme  paprika  cumin
2069    False   False    False  False  ...      True  False     True  False
74964   False   False    False  False  ...      True  False     True  False
93768    True   False    False   True  ...      True  False     True  False
113926   True   False    False  False  ...      True  False     True  False
137686   True   False    False  False  ...      True  False     True  False
140530   True   False    False  False  ...      True   True     True  False
158475   True   Fal